# Pneumonia detection from X-ray

## downloading tensorflow for the project
##### run the following command in a code sell to download tensorflow



```terminal
!pip install tensorflow
```



### transfer learning is used to make this project 
####  <u>Transfer learning</u> :   Transfer Learning is basically when you train a model on a data and then used that trained model on a diffrent but similer data so that you don't have to train the entire model , which may take a lot of time and GPU power and for a good preformance will also required a large data set, which we know may not be peresent and making our own data set is tedious and expencive task

##### we will be using VGG16 and VGG19

#### you can donload the data set form [here](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)

In [3]:
import tensorflow as tf

# Check if GPU is available and set the device
if tf.config.list_physical_devices('GPU'):
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    device_name = tf.test.gpu_device_name()
    print(f"GPU device name: {device_name}")
else:
    print("No GPU found, using CPU.")

# Example TensorFlow operations using GPU
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

print(c)

# Monitor GPU utilization
!nvidia-smi


Num GPUs Available:  2
GPU device name: /device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Wed Jul  3 15:11:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              27W /  70W |  14065MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
# importing all the necessary libraries
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential 
import numpy as np
import glob as glob
import matplotlib.pyplot as plt

In [24]:
# re-sizing all the images ,as we are delaing with rgb images so it will have 3 dimentions
IMAGE_SIZE = (224, 224)

train_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
valid_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

In [56]:
# using vgg16 model , and we will be using imagenet weights, as they were the one who win the competion and so ther weights 
# are of the trained modle so we won't have to train the whole model , this is transfer learnig 

from tensorflow.keras.applications import ResNet50

# Define the model with pre-trained weights from COCO
resNet = ResNet50(input_shape=IMAGE_SIZE.append((3)), weights='imagenet', include_top=True) 
# [3] is for the dimention of the rgb imgae

# include_top = False means we don't want the exact model as we are going to use a diffrent data set so , we will train 
# the the last layer and the output layer for out dataset so that the model predicts for this particular data set
# in the resNet50 starting layers of the cnn identify the basic pattrens that are persent in all the images like lines, edges etc, 
# so we don't need to change the staring layers for that we have keept them as same and removed the top part that we will be 
# training for out data set

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [81]:
# we are loading a pretrained model so we don't want to change the weights that we have laoded in the avobe code cell 
for layer in resNet.layers:
        layer.trainable = False    # so that we don't train the layers

In [82]:
from glob import glob
folders = glob("/kaggle/input/chest-xray-pneumonia/chest_xray/train/*")

# this will tell us which folders are present in Data_set/train , * repersents all , 

In [83]:
folders

['/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA',
 '/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL']

In [84]:
from tensorflow.keras.layers import Dropout
x = Flatten()(resNet.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
# here flatten is bsically reducing the dimentionality of the layers , making it in 1D
# vgg.output is the output layer and we are flattening the output layre

In [85]:
prediction = Dense(len(folders), activation= "softmax")(x)
# x is our output layer and we are usign Dense function to five it no. of neurons that there are in the train file
# for this case there are only two , pneumonia and not pneumonia 
# we are using activatin function softmax

model = Model(inputs = resNet.input, outputs = prediction)
# we are giving the model the inputs that are pretrained form vgg and outputs according to out data set

In [86]:
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 26,281,066 (100.25 MB)

 Trainable params: 644,354 (2.46 MB)

 Non-trainable params: 25,636,712 (97.80 MB)

In [94]:
# now we will compile the model , i.e. giving it all the things it will need to train well and make good predictions 
from tensorflow.keras.optimizers import Adam
model.compile(loss = 'categorical_crossentropy', 
             optimizer = Adam(0.0001),
             metrics = ["accuracy"])

In [88]:
# using image generator to import the images form the data set


train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2, 
                                  horizontal_flip=True)

# here we are configuring the ImageDataGenerator with the parameters like rescale and all so that our model performs well
# it is adviced that if all the data is in same range then its better for the model 
### the basic reason we are doing this is because we want to increase out tring data somehow so that our model doesn't overfit
### this data augmentation is helping us with generarating more data for the training 

# rescale = 1./255 is rescaling the image's pixel value that ranges form 0 to 255 by deviding by 255 we can make it in range of 
# [0,1], this is a convention for better training 

# shear_range=0.2 shearing means keeping one part of the image fixed and move the other part in one direction keeping the 
# image stationary, this can introduce diversity into the training data

# horizontal_flip is used to mirror image this helps in making the model invarient to horizontal flips


test_datagen = ImageDataGenerator(rescale = 1./255)

In [89]:
# training data
# make sure to provide the same target size as initialized before 
training_set = train_datagen.flow_from_directory("/kaggle/input/chest-xray-pneumonia/chest_xray/train",
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode= "categorical")

# at once 32 images will be given for training and we have to categorize the images 
# target size is (244, 244) this is done to ensure that the size of all the images is same when being fed to the model


Found 5216 images belonging to 2 classes.


In [90]:
# doing the same for test data
test_set = test_datagen.flow_from_directory("/kaggle/input/chest-xray-pneumonia/chest_xray/test",
                                                target_size=(224,224),
                                                batch_size=32,
                                                class_mode= "categorical")

Found 624 images belonging to 2 classes.


In [91]:
print(len(test_set))
print(next(iter(test_set))[0].shape, next(iter(test_set))[1].shape)

20
(32, 224, 224, 3) (32, 2)


In [95]:
# now we train our model 
xray = model.fit(
    training_set,
    validation_data=test_set,
    epochs=25,
    steps_per_epoch=len(training_set),
    validation_steps=(len(test_set)-1)
)

Epoch 1/25
  2/163 ━━━━━━━━━━━━━━━━━━━━ 19s 119ms/step - accuracy: 0.6094 - loss: 0.6909  

W0000 00:00:1720033514.392122     140 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 634ms/step - accuracy: 0.7393 - loss: 0.6481

W0000 00:00:1720033624.498938     140 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


163/163 ━━━━━━━━━━━━━━━━━━━━ 148s 717ms/step - accuracy: 0.7393 - loss: 0.6479 - val_accuracy: 0.6266 - val_loss: 0.6879
Epoch 2/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 7s 44ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5625 - val_loss: 0.7531
Epoch 3/25


W0000 00:00:1720033637.652898     138 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


163/163 ━━━━━━━━━━━━━━━━━━━━ 111s 647ms/step - accuracy: 0.7445 - loss: 0.5691 - val_accuracy: 0.6234 - val_loss: 0.6889
Epoch 4/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6875 - val_loss: 0.6256
Epoch 5/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 115s 672ms/step - accuracy: 0.7422 - loss: 0.5740 - val_accuracy: 0.6234 - val_loss: 0.6909
Epoch 6/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6875 - val_loss: 0.6252
Epoch 7/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 116s 686ms/step - accuracy: 0.7435 - loss: 0.5698 - val_accuracy: 0.6250 - val_loss: 0.6946
Epoch 8/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6250 - val_loss: 0.6933
Epoch 9/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 637ms/step - accuracy: 0.7431 - loss: 0.5685 - val_accuracy: 0.6217 - val_loss: 0.6969
Epoch 10/25
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 666us/step - accuracy: 0.00

In [96]:
# from keras.models import load_model
model.save("/kaggle/working/ResNet_pneumonia_epoch_25.h5")

In [2]:
from keras.models import load_model
loaded_model = load_model("VGG16_pneumonia.h5")

In [99]:
# image has to be converted into numbers --> array to before to be processed

img = image.load_img( "/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1436-0001.jpeg", target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(img, axis = 0)
img_data = preprocess_input(x)
ans = model.predict(img_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


In [100]:
predicted_class = np.argmax(ans)
predicted_class

1

In [101]:
if predicted_class == 0:
    print("you dont have Pneumonia")
else:
    print("Sorry, you have Pneumonia")

Sorry, you have Pneumonia
